# Setup

In [ ]:
# Parameters
bs = None
keyframes_dir = None
save_dir = None

In [ ]:
import os

dir_path = os.getcwd()

if not keyframes_dir:
    if 'google.colab' in str(get_ipython()):
        # Update this path as necessary
        keyframes_dir = f'{dir_path}/Keyframes'
    elif 'kaggle' in str(get_ipython()):
        keyframes_dir = f'{dir_path}/Keyframes'
    else:
        parent_dir_path = os.path.dirname(dir_path)
        keyframes_dir = f'{parent_dir_path}/transnet/Keyframes'
        
if not bs:
    bs = 16
    
if not save_dir:
    save_dir = './ocr'

In [ ]:
! pip install easyocr

In [ ]:
import os
import glob
import json
import easyocr
from tqdm import tqdm

# Parse data path

In [ ]:
def parse_keyframe_info(keyframes_dir = '../transnet/Keyframes'):
    all_keyframe_paths = {}
    for part in sorted(os.listdir(keyframes_dir)):
        data_part = part.split('/')[-1]
        all_keyframe_paths[data_part] = {}
        
    for data_part in sorted(all_keyframe_paths.keys()):
        data_part_path = f'{keyframes_dir}/{data_part}'
        video_dirs = sorted(os.listdir(data_part_path))
        video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
        for video_id, video_dir in zip(video_ids, video_dirs):
            keyframe_paths = sorted(
                glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
            all_keyframe_paths[data_part][video_id] = keyframe_paths
    
    return all_keyframe_paths

# Inference

In [ ]:
def create_directory(path):
    """Create a directory if it does not exist."""
    if not os.path.exists(path):
        os.makedirs(path)


def save_ocr_results(save_dir, key, video_id, ocr_results):
    """Save OCR results to a JSON file."""
    with open(os.path.join(save_dir, key, f"{video_id}.json"), "w", encoding='utf-8') as jsonfile:
        json.dump(ocr_results, jsonfile, ensure_ascii=False, indent=4)


def process_video_keyframes(reader, video_keyframe_paths, batch_size=16):
    """Process keyframes of a video and perform OCR."""
    video_ocr_results = {}

    for i in range(0, len(video_keyframe_paths), batch_size):
        image_paths = video_keyframe_paths[i:i+batch_size]
        results = reader.readtext_batched(
            image_paths, batch_size=len(image_paths))
        for image_path, result in zip(image_paths, results):
            frame_name = os.path.basename(image_path)
            refined_result = [item for item in result if item[2] > 0.5]
            refined_result = easyocr.utils.get_paragraph(refined_result)
            text_detected = [item[1] for item in refined_result]
            video_ocr_results[frame_name] = text_detected
    return video_ocr_results


def ocr_and_save_results(reader, all_keyframe_paths, save_dir, batch_size=16):
    """Perform OCR on keyframes and save results to JSON files."""
    create_directory(save_dir)
    keys = sorted(all_keyframe_paths.keys())

    for key in tqdm(keys, desc="Processing keys"):
        video_keyframe_paths = all_keyframe_paths[key]
        video_ids = sorted(video_keyframe_paths.keys())
        key_dir = os.path.join(save_dir, key)
        create_directory(key_dir)

        for video_id in tqdm(video_ids, desc=f"Processing {key}"):
            video_keyframe_path = video_keyframe_paths[video_id]
            video_ocr_results = process_video_keyframes(
                reader, video_keyframe_path, batch_size)
            save_ocr_results(save_dir, key, video_id, video_ocr_results)

In [ ]:
all_keyframe_paths = parse_keyframe_info(keyframes_dir)
# this needs to run only once to load the model into memory
reader = easyocr.Reader(['vi'], gpu=True)
ocr_and_save_results(reader, all_keyframe_paths, save_dir, bs)

Processing keys: 100%|██████████| 2/2 [02:13<00:00, 66.73s/it]
